In [1]:
import pandas as pd
import numpy as np
import os
import sys
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
import re
from sklearn.preprocessing import RobustScaler
import scipy.stats as ss
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
import scipy.io
from __future__ import division
from __future__ import print_function
import os
import sys
from time import time
import scipy.stats as ss

# temporary solution for relative imports in case pyod is not installed
# if pyod is installed, no need to use the following line
sys.path.append(
    os.path.abspath(os.path.join(os.path.dirname("__file__"), '..')))
from numpy import percentile
import matplotlib.pyplot as plt
import matplotlib.font_manager
# Import all models
from pyod.models.abod import ABOD
from pyod.models.cblof import CBLOF
from pyod.models.feature_bagging import FeatureBagging
from pyod.models.hbos import HBOS
from pyod.models.iforest import IForest
from pyod.models.knn import KNN
from pyod.models.lof import LOF
from pyod.models.mcd import MCD
from pyod.models.ocsvm import OCSVM
from pyod.models.pca import PCA
from pyod.models.lscp import LSCP # ensemble

In [2]:
# dataset path
data_path = "/Users/kadima/experiment_any/anomaly-detection/datasets/"

# load data

In [20]:
def getData(fileName):
    # check fileName:
    files = [x for x in os.listdir(data_path) if x.endswith(".mat")]
    mat = scipy.io.loadmat(data_path+fileName)
    X = mat["X"]
    y = mat["y"]
    return X, y

def read_data_parkinson():
    file_path = "/Users/kadima/experiment_any/anomaly-detection/Parkinson_withoutdupl_75.txt"
    with open(file_path,'r') as f:
        data = f.readlines()
    # print(data[28:-2])
    col_names = []
    list_ = []
    for id_, line in enumerate(data):
        if line.startswith("@ATTRIBUTE"):
            colName = re.findall("@ATTRIBUTE '(.*?)'", line)[0]
            col_names.append(colName)
        if id_ >= 28 and id_<= len(data) - 1 and line != "\n":
            new_line = line.split(",")
            new_line = [x.strip() for x in new_line]
            new_line = [float(new_line[x]) for x in range(len(new_line)-1)] + \
                       [new_line[-1].replace("'", "")]
            list_.append(new_line)

    df = pd.DataFrame(list_, columns=col_names)
    df['outlier'] = df.outlier.apply(lambda x: 1 if x =='yes' else 0)
    X = df.iloc[:, 1:-1]
    y = df.iloc[:, -1]
    return X, y




# thresholders

In [92]:
def sd_thresholder(scores, real_y, factor=2.5):
    high_limit = np.mean(scores) + factor*np.std(scores)
    y_predict = scores>=high_limit
    y_predict = [1 if j else 0 for j in y_predict]
    f1 = f1_score(real_y, y_predict)
    return y_predict, f1


def mad_thresholder(scores, real_y):
    median_ = np.median(scores)
    mad = 1.4826*np.median(np.abs(scores-median_))
    y_predict = scores>= 3*mad
    y_predict = [1 if j else 0 for j in y_predict]
    f1 = f1_score(real_y, y_predict)
    return y_predict, f1

def iqr_thresholder(scores, real_y):
    iqr = np.percentile(scores,75) - np.percentile(scores,25)
    y_predict = scores >= (np.percentile(scores,75)+1.5*iqr)
    y_predict = [1 if j else 0 for j in y_predict]
    f1 = f1_score(real_y, y_predict)
    return y_predict, f1

def disagreement(score_matrix, num_detectors, real_y):
    rank_matrix = np.zeros([len(score_matrix), num_detectors])
    for i in range(num_detectors):
    # rank by each column and get its rank_position
        rank_matrix[:,i] = ss.rankdata(score_matrix[:, i])
    std_record = np.zeros(len(X))
    for i in range(len(score_matrix)):
        std_record[i] = np.std(rank_matrix[i,:])
    # get the union indexs of largest 50% rank
    vdi = np.mean()
    
    std_max_rows = np.argsort(std_record)[-1:]
    std_median_scores = np.median(score_matrix[std_max_rows,:], axis=0)
    threshold_for_each_detector = std_median_scores
    
    f1_list = []
    for i in range(num_detectors):
        outliers_rows = score_matrix[:,i] >= threshold_for_each_detector[i]
        y_predict = [1 if j else 0 for j in outliers_rows]
        f1 = f1_score(real_y, y_predict)
        f1_list.append(f1)
    return f1_list


In [93]:
def get_score_matrix(X, num_detectors):
    return np.zeros([X.shape[0], num_detectors])

def get_perform_matrix(num_thresholders, num_detectors):
    return np.zeros((num_thresholders, num_detectors))

# Detectors

In [94]:
random_state = np.random.RandomState(10)
outliers_fraction = 0.4
# initialize a set of detectors for LSCP
detector_list = [LOF(n_neighbors=5), LOF(n_neighbors=10), LOF(n_neighbors=15),
                 LOF(n_neighbors=20), LOF(n_neighbors=25), LOF(n_neighbors=30),
                 LOF(n_neighbors=35), LOF(n_neighbors=40), LOF(n_neighbors=45),
                 LOF(n_neighbors=50)]
classifiers = {
    'Angle-based Outlier Detector (ABOD)':
        ABOD(contamination=outliers_fraction),
    'Cluster-based Local Outlier Factor (CBLOF)':
        CBLOF(contamination=outliers_fraction,
              check_estimator=False, random_state=random_state),
    'Feature Bagging':
        FeatureBagging(LOF(n_neighbors=35),
                       contamination=outliers_fraction,
                       random_state=random_state),
    'Histogram-base Outlier Detection (HBOS)': HBOS(
        contamination=outliers_fraction),
    'Isolation Forest': IForest(contamination=outliers_fraction,
                                random_state=random_state, n_estimators = 200),
    'K Nearest Neighbors (KNN)': KNN(
        contamination=outliers_fraction),
    'Average KNN': KNN(method='mean',
                       contamination=outliers_fraction),
    'Local Outlier Factor (LOF)':
        LOF(n_neighbors=35, contamination=outliers_fraction),
    'Minimum Covariance Determinant (MCD)': MCD(
        contamination=outliers_fraction, random_state=random_state),
    'One-class SVM (OCSVM)': OCSVM(contamination=outliers_fraction),
    'Principal Component Analysis (PCA)': PCA(
        contamination=outliers_fraction, random_state=random_state),
    'Locally Selective Combination (LSCP)': LSCP(
        detector_list, contamination=outliers_fraction,
        random_state=random_state)
}

names = []
# Show all detectors
for i, clf in enumerate(classifiers.keys()):
    names.append(clf)
#     print('Model', i + 1, clf)

In [95]:
X,y = read_data_parkinson()

In [96]:
# Fit the models with the generated data and 
# compare model performances

def get_result(X, y, classifiers):
    # create matrix to store the performance
    score_matrix = get_score_matrix(X, len(classifiers.keys()))
    perform_table = get_perform_matrix(5, len(classifiers.keys()) )

    np.random.seed(5)
    clfs = []
    # Fit the model
    for i, (clf_name, clf) in enumerate(classifiers.items()):
        print(i + 1, 'fitting', clf_name)
        # fit the data and tag outliers
        clf.fit(X)
        clfs.append(clf)
        scores_pred = clf.decision_function(X)
        score_matrix[:, i] = scores_pred

    for i, thresholder in enumerate([sd_thresholder, mad_thresholder, 
                                     iqr_thresholder]):
        for j in range(score_matrix.shape[1]):
            perform_table[i,j] = thresholder(score_matrix[:,j], y)[1]
            
    for i in range(score_matrix.shape[1]):
        perform_table[-2,i] = f1_score(y,clfs[i].predict(X))
        
    perform_table[-1,:] = disagreement(score_matrix, len(classifiers), y)
    
    return pd.DataFrame(perform_table, columns = names, index = ["sd",'mad','iqr','default','disagreement'])

In [97]:
df = get_result(X, y, classifiers)

1 fitting Angle-based Outlier Detector (ABOD)
2 fitting Cluster-based Local Outlier Factor (CBLOF)
3 fitting Feature Bagging
4 fitting Histogram-base Outlier Detection (HBOS)
5 fitting Isolation Forest


/opt/anaconda3/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:792: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 1.0 (renaming of 0.25).
  warnings.warn("'n_jobs' was deprecated in version 0.23 and will be"


6 fitting K Nearest Neighbors (KNN)
7 fitting Average KNN
8 fitting Local Outlier Factor (LOF)
9 fitting Minimum Covariance Determinant (MCD)
10 fitting One-class SVM (OCSVM)
11 fitting Principal Component Analysis (PCA)
12 fitting Locally Selective Combination (LSCP)


/opt/anaconda3/lib/python3.8/site-packages/sklearn/covariance/_robust_covariance.py:647: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn("The covariance matrix associated to your dataset "


In [98]:
df

,Angle-based Outlier Detector (ABOD),Cluster-based Local Outlier Factor (CBLOF),Feature Bagging,Histogram-base Outlier Detection (HBOS),Isolation Forest,K Nearest Neighbors (KNN),Average KNN,Local Outlier Factor (LOF),Minimum Covariance Determinant (MCD),One-class SVM (OCSVM),Principal Component Analysis (PCA),Locally Selective Combination (LSCP)
sd,0.000000,0.101911,0.101911,0.103226,0.090909,0.051948,0.065359,0.101911,0.040000,0.000000,0.065789,0.089744
mad,0.000000,0.176471,0.859649,0.000000,0.172840,0.243094,0.243094,0.859649,0.573991,0.026490,0.676259,0.134969
iqr,0.000000,0.125000,0.113208,0.196319,0.184049,0.147239,0.145455,0.113208,0.255814,0.000000,0.115385,0.135802
default,0.526316,0.515556,0.500000,0.657778,0.524444,0.411483,0.328205,0.479638,0.586667,0.524444,0.453333,0.466368
disagreement,0.844311,0.844311,0.316940,0.764479,0.744828,0.826748,0.849558,0.254335,0.844720,0.673993,0.644444,0.443350


In [99]:
df.apply(lambda x: np.mean(x), axis=1)

sd              0.067726
mad             0.330542
iqr             0.127623
default         0.497853
disagreement    0.671001
dtype: float64

In [100]:
result_dict = dict()
for datasets in ['lympho.mat','wbc.mat']:
    X,y = getData(datasets)
    result_dict[datasets] = get_result(X, y, classifiers)

1 fitting Angle-based Outlier Detector (ABOD)
2 fitting Cluster-based Local Outlier Factor (CBLOF)
3 fitting Feature Bagging
4 fitting Histogram-base Outlier Detection (HBOS)
5 fitting Isolation Forest


/opt/anaconda3/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:792: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 1.0 (renaming of 0.25).
  warnings.warn("'n_jobs' was deprecated in version 0.23 and will be"


6 fitting K Nearest Neighbors (KNN)
7 fitting Average KNN
8 fitting Local Outlier Factor (LOF)
9 fitting Minimum Covariance Determinant (MCD)
10 fitting One-class SVM (OCSVM)
11 fitting Principal Component Analysis (PCA)
12 fitting Locally Selective Combination (LSCP)
1 fitting Angle-based Outlier Detector (ABOD)
2 fitting Cluster-based Local Outlier Factor (CBLOF)
3 fitting Feature Bagging


/opt/anaconda3/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:792: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 1.0 (renaming of 0.25).
  warnings.warn("'n_jobs' was deprecated in version 0.23 and will be"


4 fitting Histogram-base Outlier Detection (HBOS)
5 fitting Isolation Forest
6 fitting K Nearest Neighbors (KNN)
7 fitting Average KNN
8 fitting Local Outlier Factor (LOF)
9 fitting Minimum Covariance Determinant (MCD)
10 fitting One-class SVM (OCSVM)
11 fitting Principal Component Analysis (PCA)
12 fitting Locally Selective Combination (LSCP)


In [101]:
result_dict['lympho.mat']

,Angle-based Outlier Detector (ABOD),Cluster-based Local Outlier Factor (CBLOF),Feature Bagging,Histogram-base Outlier Detection (HBOS),Isolation Forest,K Nearest Neighbors (KNN),Average KNN,Local Outlier Factor (LOF),Minimum Covariance Determinant (MCD),One-class SVM (OCSVM),Principal Component Analysis (PCA),Locally Selective Combination (LSCP)
sd,0.000000,0.400000,0.909091,0.800000,0.909091,0.600000,0.600000,0.666667,0.500000,0.400000,0.666667,0.666667
mad,0.000000,0.077922,0.077922,0.000000,0.909091,0.077922,0.077922,0.077922,0.153846,0.545455,0.077922,0.545455
iqr,0.000000,0.375000,0.750000,0.857143,0.923077,0.500000,0.533333,0.666667,0.500000,0.545455,0.521739,0.750000
default,0.166667,0.184615,0.193548,0.184615,0.184615,0.342857,0.500000,0.184615,0.184615,0.184615,0.184615,0.181818
disagreement,0.096000,0.085714,0.307692,0.222222,0.244898,0.092308,0.107143,0.255319,0.145455,0.279070,0.375000,0.214286


In [102]:
result_dict['lympho.mat'].apply(lambda x: np.mean(x), axis=1)

sd              0.593182
mad             0.218448
iqr             0.576868
default         0.223100
disagreement    0.202092
dtype: float64

In [103]:
result_dict['wbc.mat'].apply(lambda x: np.mean(x), axis=1)

sd              0.412609
mad             0.238732
iqr             0.478953
default         0.256351
disagreement    0.231884
dtype: float64

In [ ]:
def em_thresholder(score_matrix, num_detectors, real_y):
    # define disagreement
    